In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import requests 
import json

`Hits` - кол-во уникальных запросов одного пользователя <br>
`Sess` - кол-во сессий пользователя <br>
`SQ` -  кол-во сессий, когда по данному запросу была найдена данная (sku) <br>
`HQ` - сколько раз показывался товар по данному запросу (sku)<br>
`metric` - метрика, может быть: добавление товара в корзину, кол-во просмотров <br>
    1. `ca_search_from_list` - добавление в корзину из поиска 
    2. `ca_search_from_pdp` - добавления в корзину из карточки после поиска 
    3. `impression` - показы 
    4. `pdp_view` - просмотры 

In [83]:
final_dataset = pd.read_csv('suggest_data.csv')

/home/mkrupin/git-folders/tools/python/suggest-prediction/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [86]:
final_dataset.head()

,id,query,metric,hits,sessions
0,31468448,анна тодд,pdp_view,50,37
1,19874893,свинтус,pdp_view,180,146
2,142125702,шоколад,pdp_view,20,2
3,142106214,косметика детская,pdp_view,6,2
4,147077350,сергей тармашев,pdp_view,8,6


In [137]:
class SuggestPrediction:
    
    
    def findSuggests(self, jsn):

        for i in jsn:
            if i['type'] == 'suggests':
                return i


    def get_better_suggestion(self, query):

        url = 'http://searchteam-suggest-api.osrch.prod.s.o3.ru/suggest?text={0}'.format(query)
        sug_req = json.loads(requests.get(url).text)['completions']

       
        sug_req = self.findSuggests(sug_req)['completion_values']

        all_suggestion = [i['text'] for i in sug_req]
        
        if query not in all_suggestion:
            all_suggestion.append(query)

        best_good = pd.DataFrame(columns=final_dataset.columns)
        for i in all_suggestion:

            main_query = final_dataset[final_dataset['query'] == i] 

            if len(main_query) != 0:
                search = main_query[final_dataset['metric'] == 'ca_search_from_list'].sort_values(by='hits',ascending=False).iloc[0]
                if search.shape == 0:
                    search = main_query[final_dataset['metric'] == 'ca_search_from_pdp'].sort_values(by='hits',ascending=False).iloc[0]
                elif search.shape == 0:
                    search = main_query[final_dataset['metric'] == 'pdp_view'].sort_values(by='hits',ascending=False).iloc[0]

                best_good = best_good.append(search)

                
            else:
                f_id = [] 

        return list(best_good['id'])
    
top_goods = SuggestPrediction().get_better_suggestion('часы')
print('Top SKU of goods:',top_goods)

Top SKU of goods: [145123330, '144072677', 142877014, 147156478, 145076482]
